In [22]:
from selenium import webdriver
from pyshadow.main import Shadow
from selenium.webdriver.common.keys import Keys
import re
import time
import string

FILE = "american-english.txt"
ALLOWABLE_CHARACTERS = set(string.ascii_letters)
ALLOWED_ATTEMPTS = 6
WORD_LENGTH = 5

In [23]:
wordlist = [word.strip() for word in open(FILE, "r").readlines()]

#creating a set of valid words according to the rules of Wordle
WORDS = {
    word.lower()
    for word in wordlist    
    if len(word) == WORD_LENGTH and set(word) < ALLOWABLE_CHARACTERS
    }
WORDS

{'ralph',
 'optic',
 'shown',
 'gauge',
 'junto',
 'drain',
 'retro',
 'draft',
 'relic',
 'louse',
 'pagan',
 'curly',
 'below',
 'brace',
 'known',
 'macho',
 'squat',
 'reach',
 'aisle',
 'artsy',
 'laugh',
 'tepee',
 'zonal',
 'gorge',
 'burnt',
 'inner',
 'hedge',
 'itchy',
 'ounce',
 'petty',
 'ivory',
 'horny',
 'polyp',
 'siren',
 'blond',
 'youth',
 'droop',
 'voter',
 'catty',
 'canon',
 'taboo',
 'golem',
 'pupil',
 'roast',
 'rearm',
 'relax',
 'touch',
 'bravo',
 'onion',
 'sigma',
 'fluid',
 'scarf',
 'store',
 'boney',
 'cable',
 'chase',
 'atone',
 'arbor',
 'vocal',
 'drive',
 'beard',
 'musky',
 'drake',
 'probe',
 'dwelt',
 'geese',
 'leafy',
 'ascot',
 'olden',
 'hyper',
 'egret',
 'gamut',
 'swath',
 'bowel',
 'pearl',
 'shook',
 'faint',
 'axion',
 'roger',
 'flume',
 'sugar',
 'uncut',
 'lemur',
 'mania',
 'blank',
 'tried',
 'allay',
 'globe',
 'abled',
 'uncle',
 'spelt',
 'thyme',
 'bulky',
 'agape',
 'shine',
 'tidal',
 'dirge',
 'edict',
 'shark',
 'vivid',


In [24]:
from collections import Counter
from itertools import chain

#Creating a Counter Object that counts the number of occurences
#of each letter used across all valid Wordle words

LETTER_COUNTER = Counter(chain.from_iterable(WORDS))

LETTER_COUNTER


Counter({'r': 899,
         'a': 979,
         'l': 719,
         'p': 367,
         'h': 389,
         'o': 754,
         't': 729,
         'i': 671,
         'c': 477,
         's': 669,
         'w': 195,
         'n': 575,
         'g': 311,
         'u': 467,
         'e': 1233,
         'j': 27,
         'd': 393,
         'f': 230,
         'y': 425,
         'b': 281,
         'k': 210,
         'm': 316,
         'q': 29,
         'z': 40,
         'v': 153,
         'x': 37})

In [25]:
#finding the total occurances of all letters
total = len(WORDS)*WORD_LENGTH
LETTER_FREQUENCY = {character: value / total 
                    for character, value in LETTER_COUNTER.items()}

LETTER_FREQUENCY

{'r': 0.07766738660907127,
 'a': 0.08457883369330453,
 'l': 0.06211663066954644,
 'p': 0.031706263498920084,
 'h': 0.033606911447084234,
 'o': 0.0651403887688985,
 't': 0.06298056155507559,
 'i': 0.057969762419006476,
 'c': 0.04120950323974082,
 's': 0.057796976241900645,
 'w': 0.016846652267818573,
 'n': 0.04967602591792657,
 'g': 0.026868250539956805,
 'u': 0.04034557235421166,
 'e': 0.10652267818574514,
 'j': 0.002332613390928726,
 'd': 0.033952483801295896,
 'f': 0.019870410367170625,
 'y': 0.0367170626349892,
 'b': 0.02427645788336933,
 'k': 0.01814254859611231,
 'm': 0.027300215982721383,
 'q': 0.002505399568034557,
 'z': 0.0034557235421166306,
 'v': 0.013218142548596112,
 'x': 0.0031965442764578834}

In [26]:
def calculate_word_commonality(word):
    score = 0
    for char in word:
        score += LETTER_FREQUENCY[char]
    return score / (WORD_LENGTH - len(set(word)) + 1)

In [27]:
import operator

def sort_by_word_commonality(words):
    sort_by = operator.itemgetter(1)
    return sorted(
        [(word, calculate_word_commonality(word)) for word in words],
        key = sort_by,
        reverse=True,
    )

def display_word_table(word_commonalities):
    for (word,freq) in word_commonalities:
        print(f"{word:<10} | {freq:<5.2}")


In [28]:
def input_word():
    while True:
        word = input("Input the word you entered> ")
        if len(word) == WORD_LENGTH and word.lower() in WORDS:
            break
    return word.lower()

def input_response():
    print("Type the color-coded reply from Wordle:")
    print(" G for Green")
    print(" Y for Yellow")
    print(" ? for Gray")
    while True:
        response = input("Response from Wordle> ")
        if len(response) == WORD_LENGTH and set(response) <= {"G", "Y", "?"}:
            break
        else:
            print(f"Error - invalid answer {response}")
    return response

In [29]:
def match_word_vector(word, word_vector):
    assert len(word) == len(word_vector)
    for letter, v_letter in zip(word, word_vector):
        if letter not in v_letter:
            return False
    return True

def match(word_vector, possible_words):
    return [word for word in possible_words if match_word_vector(word, word_vector)]

In [30]:
def enter_word(browser, word):
    shadow = Shadow(browser)
    wordleBot = browser.find_element_by_tag_name('html')
    wordleBot.click()
    time.sleep(1)
    wordleBot.send_keys(word)
    wordleBot.send_keys(Keys.ENTER)
    time.sleep(1)
    finalEvaluation = list(word)
    gameRow = shadow.find_element("game-row[letters="+word+"]")
    for char in set(word):
        gameTile = shadow.find_elements(gameRow,"game-tile[letter=" + char +"]")
        index = -1
        for elem in gameTile:
            if elem.get_attribute("evaluation") == 'present':
                finalEvaluation[word.index(char, index + 1, len(word))] = 'Y' 
            if elem.get_attribute("evaluation") == 'correct':
                finalEvaluation[word.index(char, index + 1, len(word))] = 'G'
            if elem.get_attribute("evaluation") == 'absent':
                finalEvaluation[word.index(char, index + 1, len(word))] = '?'
            index = word.index(char, index + 1, len(word))
    return finalEvaluation

In [31]:
def solve():
    browser = webdriver.Chrome(executable_path ='/usr/local/bin/chromedriver')
    shadow = Shadow(browser)
    browser.get('http://www.powerlanguage.co.uk/wordle/')
    time.sleep(1)
    possible_words = WORDS.copy()
    word_vector = [set(string.ascii_lowercase) for _ in range(WORD_LENGTH)]
    yLetters = set()
    for attempt in range(1, ALLOWED_ATTEMPTS + 1):
        print(f"Attempt {attempt} with {len(possible_words)} possible words")
        sortedWords = sort_by_word_commonality(possible_words)
        display_word_table(sortedWords[:15])
        i = 0
        word = sortedWords[0][0]
        while yLetters.issubset(set(word)) == False:
            i = i + 1
            word = sortedWords[i][0]
        response = enter_word(browser,word)
        for idx, letter in enumerate(response):
            if letter == "G":
                word_vector[idx] = {word[idx]}
            elif letter == "Y":
                try:
                    word_vector[idx].remove(word[idx])
                    yLetters.add(word[idx])
                except KeyError:
                    pass
            elif letter == "?":
                for vector in word_vector:
                    try:
                        vector.remove(word[idx])
                    except KeyError:
                        pass
        possible_words = match(word_vector, possible_words)
    

In [32]:
solve()

<ipython-input-31-fa2af271ee1b>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Chrome(executable_path ='/usr/local/bin/chromedriver')


Attempt 1 with 2315 possible words
later      | 0.39 
alter      | 0.39 
alert      | 0.39 
arose      | 0.39 
irate      | 0.39 
stare      | 0.39 
raise      | 0.38 
arise      | 0.38 
renal      | 0.38 
learn      | 0.38 
saner      | 0.38 
snare      | 0.38 
steal      | 0.37 
slate      | 0.37 
least      | 0.37 


<ipython-input-30-51aad33ca172>:3: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  wordleBot = browser.find_element_by_tag_name('html')


QA--QAQA True
Attempt 2 with 304 possible words
scorn      | 0.29 
irony      | 0.29 
rhino      | 0.28 
shorn      | 0.28 
curio      | 0.28 
groin      | 0.28 
robin      | 0.27 
scion      | 0.27 
sonic      | 0.27 
crony      | 0.27 
corny      | 0.27 
micro      | 0.27 
noisy      | 0.27 
sworn      | 0.27 
round      | 0.27 
QA--QAQA True
Attempt 3 with 4 possible words
robin      | 0.27 
moron      | 0.14 
union      | 0.13 
onion      | 0.096
QA--QAQA True
Attempt 4 with 1 possible words
robin      | 0.27 
QA--QAQA True
Attempt 5 with 1 possible words
robin      | 0.27 
QA--QAQA True
Attempt 6 with 1 possible words
robin      | 0.27 
QA--QAQA True
